In [1]:
import requests
import json
import gradio as gr
import json
import requests
import os
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from openai import OpenAI
GPT_MODEL = "gpt-3.5-turbo"

client = OpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
)

d:\python_env\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'termcolor'

{
  "results": [{
    "location": {
      "id": "WX4FBXXFKE4F",
      "name": "北京",
      "country": "CN",
      "path": "北京,北京,中国",
      "timezone": "Asia/Shanghai",
      "timezone_offset": "+08:00"
    },
    "daily": [{                          //返回指定days天数的结果
      "date": "2015-09-20",              //日期（该城市的本地时间）
      "text_day": "多云",                //白天天气现象文字
      "code_day": "4",                  //白天天气现象代码
      "text_night": "晴",               //晚间天气现象文字
      "code_night": "0",                //晚间天气现象代码
      "high": "26",                     //当天最高温度
      "low": "17",                      //当天最低温度
      "precip": "0",                    //降水概率，范围0~1，单位百分比（目前仅支持国外城市）
      "wind_direction": "",             //风向文字
      "wind_direction_degree": "255",   //风向角度，范围0~360
      "wind_speed": "9.66",             //风速，单位km/h（当unit=c时）、mph（当unit=f时）
      "wind_scale": "",                 //风力等级
      "rainfall": "0.0",                //降水量，单位mm
      "humidity": "76"                  //相对湿度，0~100，单位为百分比
    }, {
      "date": "2015-09-21",
      "text_day": "晴",
      "code_day": "0",
      "text_night": "晴",
      "code_night": "0",
      "high": "27",
      "low": "17",
      "precip": "0",
      "wind_direction": "",
      "wind_direction_degree": "157",
      "wind_speed": "17.7",
      "wind_scale": "3",
      "rainfall": "0.0",
      "humidity": "76"
    }, {
      ...                               //更多返回结果
    }],
    "last_update": "2015-09-20T18:00:00+08:00" //数据更新时间（该城市的本地时间）
  }]
}

In [31]:
#调用心知天气预报API weatherResponse.json()['results'][0]['now']['temperature']
def get_current_weather(location:str) -> str:
    requestsUrl = f"https://api.seniverse.com/v3/weather/now.json?key=SUaEVEcUHIhRwQROw&location={location}&language=zh-Hans&unit=c"
    print('requestsUrl:',requestsUrl)
    weatherResponse = requests.get(requestsUrl).json()['results'][0]
    print(weatherResponse,weatherResponse["location"]["name"])
    weather_info = {
        "location": weatherResponse["location"]["name"],
        "temperature":weatherResponse["now"]["temperature"],
        "forecast": weatherResponse["now"]["text"],
    }
    return json.dumps(weather_info, ensure_ascii=False)


In [32]:
get_current_weather('beijing')

requestsUrl: https://api.seniverse.com/v3/weather/now.json?key=SUaEVEcUHIhRwQROw&location=beijing&language=zh-Hans&unit=c
{'location': {'id': 'WX4FBXXFKE4F', 'name': '北京', 'country': 'CN', 'path': '北京,北京,中国', 'timezone': 'Asia/Shanghai', 'timezone_offset': '+08:00'}, 'now': {'text': '晴', 'code': '0', 'temperature': '17'}, 'last_update': '2024-03-29T11:34:19+08:00'} 北京


'{"location": "北京", "temperature": "17", "forecast": "晴"}'

In [33]:
#调用心知天气预报API
def get_n_day_weather_forecast(location:str='beijing',start:int=0,num_days:int=5):
     print(f'{location}的天气预报：{num_days}')
     requestsUrl = f"https://api.seniverse.com/v3/weather/daily.json?key=SUaEVEcUHIhRwQROw&location={location}&language=zh-Hans&unit=c&start={start}&days={num_days}"
     print(requestsUrl)
     weatherResponse = requests.get(requestsUrl).json()['results'][0]

     location = weatherResponse["location"]["name"]
    
     date = [ daily["date"] for  daily in weatherResponse["daily"]]
     print("date",date)
     forecast_day = [ daily["text_day"] for  daily in weatherResponse["daily"]]
     forecast_night = [ daily["text_night"] for  daily in weatherResponse["daily"]]
     maximum_temperature = [ daily["high"] for  daily in weatherResponse["daily"]]
     lowest_temperature = [ daily["low"] for  daily in weatherResponse["daily"]]
     weather_info ={
         "date":date, 
         "forecast_day":forecast_day, 
         "forecast_night":forecast_night, 
         "maximum_temperature":maximum_temperature, 
         "lowest_temperature":lowest_temperature, 
     }
     print("weather_info",weather_info)
     return json.dumps(weather_info, ensure_ascii=False)

In [34]:
get_n_day_weather_forecast('beijing')

beijing的天气预报：5
https://api.seniverse.com/v3/weather/daily.json?key=SUaEVEcUHIhRwQROw&location=beijing&language=zh-Hans&unit=c&start=0&days=5
date ['2024-03-29', '2024-03-30', '2024-03-31']
weather_info {'date': ['2024-03-29', '2024-03-30', '2024-03-31'], 'forecast_day': ['晴', '晴', '晴'], 'forecast_night': ['晴', '晴', '晴'], 'maximum_temperature': ['20', '21', '23'], 'lowest_temperature': ['6', '6', '7']}


'{"date": ["2024-03-29", "2024-03-30", "2024-03-31"], "forecast_day": ["晴", "晴", "晴"], "forecast_night": ["晴", "晴", "晴"], "maximum_temperature": ["20", "21", "23"], "lowest_temperature": ["6", "6", "7"]}'

In [35]:
# 定义一个名为tools的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数

# 第一个字典定义了一个名为"get_current_weather"的功能
tools  = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",  # 功能的名称
        "description": "Get the current weather",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
                "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                    },
                },
                "required": [
                    "location"
                ],  # 该功能需要的必要参数
            },
        }
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的功能
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",  # 功能的名称
        "description": "Get an N-day weather forecast",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
                "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                    },
                    "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                    }
                },
                "required": [
                    "location",
                    "num_days"
                ]  # 该功能需要的必要参数
            }
        },
    },
]

In [36]:
@retry(wait=wait_random_exponential(multiplier=1, max=60), stop=stop_after_attempt(5))
def chat_completion_request(messagelist:list,model:str=GPT_MODEL,tool_choice=None,tools=None):
    response = client.chat.completions.create(
        model=model,
        messages=messagelist,
        tools=tools,
        tool_choice=tool_choice
    )
    return response.choices[0].message    

In [65]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can answer questions.",
    },
]



In [38]:
def add_message(content):
    messages.append(content)
    return messages
    # return [messages[0]] + messages[-4:]

In [67]:


messages

[{'role': 'system',
  'content': 'You are a helpful assistant that can answer questions.'},
 {'role': 'user', 'content': '我在北京，最近一周的天气怎么样'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yX2hShGJ2AtNanxp8jNLYmeP', function=Function(arguments='{"location":"Beijing","num_days":7}', name='get_n_day_weather_forecast'), type='function')]),
 {'tool_call_id': 'call_yX2hShGJ2AtNanxp8jNLYmeP',
  'role': 'tool',
  'name': 'get_n_day_weather_forecast',
  'content': '{"date": ["2024-03-29", "2024-03-30", "2024-03-31"], "forecast_day": ["晴", "晴", "晴"], "forecast_night": ["晴", "晴", "晴"], "maximum_temperature": ["20", "21", "23"], "lowest_temperature": ["6", "6", "7"]}'}]

In [66]:
add_message({
         "role": "user",
         "content": "我在北京，最近一周的天气怎么样"
    })
response = chat_completion_request(messages,tools=tools)
print(response)
deal_conversation(response)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yX2hShGJ2AtNanxp8jNLYmeP', function=Function(arguments='{"location":"Beijing","num_days":7}', name='get_n_day_weather_forecast'), type='function')])
Beijing的天气预报：7
https://api.seniverse.com/v3/weather/daily.json?key=SUaEVEcUHIhRwQROw&location=Beijing&language=zh-Hans&unit=c&start=0&days=7
date ['2024-03-29', '2024-03-30', '2024-03-31']
weather_info {'date': ['2024-03-29', '2024-03-30', '2024-03-31'], 'forecast_day': ['晴', '晴', '晴'], 'forecast_night': ['晴', '晴', '晴'], 'maximum_temperature': ['20', '21', '23'], 'lowest_temperature': ['6', '6', '7']}


'北京最近一周的天气情况如下：\n- 3月29日白天晴，最高温度20°C，夜间最低温度6°C。\n- 3月30日白天晴，最高温度21°C，夜间最低温度6°C。\n- 3月31日白天晴，最高温度23°C，夜间最低温度7°C。'

In [47]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant that can answer questions.'},
 {'role': 'user', 'content': '我在北京，最近一周的天气怎么样'}]

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_c7IhNraOZGgWl62DD1q3aUPs', function=Function(arguments='{"location":"Beijing","num_days":7}', name='get_n_day_weather_forecast'), type='function')])


In [49]:
#执行tool_calls
def execute_tool_calls(message) -> str:
    if message.tool_calls:
                        
            available_tools = {
                    "get_n_day_weather_forecast": get_n_day_weather_forecast,
                    "get_current_weather": get_current_weather,
            }  # only one function in this example, but you can have multiple
            
            for tool_call in message.tool_calls:
                
                function_name = tool_call.function.name
                
                function_args = json.loads(tool_call.function.arguments)
                
                if function_args.get('location'):
                    function_args['location'] = function_args['location'].split(',')[0]
                
                fuction_to_call = available_tools[function_name]
                
                function_response = fuction_to_call(
                    **function_args
                )
                
                add_message(
                    {
                        "tool_call_id":tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )
            second_response = client.chat.completions.create(
                model=GPT_MODEL,
                messages=messages,
            )
            return second_response.choices[0].message.content            
    else:    
        print("No function call in message") 
            
            

In [64]:

def deal_conversation(message):
    messages.append(message)
    if message.role == "assistant" and message.tool_calls is not None:
        return execute_tool_calls(message)
    else:
        return message.content
            
    

In [42]:
#定义聊天函数
def chat(content,history):
    add_message({
         "role": "user",
         "content": content
    })
    repsonseMsg = deal_conversation(chat_completion_request(messagelist=messages,tools=tools))
    response = str(repsonseMsg)
    return response
     

In [43]:

chat(content ='我在北京，最近一周的天气怎么样',history='')

AttributeError: 'ChatCompletionMessageToolCall' object has no attribute 'name'

In [ ]:


demo = gr.ChatInterface(fn=chat, 
                        # examples=["hello", "hola", "merhaba"],
                        title="聊天机器人",
                        retry_btn=None,
                        undo_btn=None,
                        clear_btn=None,
                        )
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [68]:
# messages
chat('我在北京，我想知道最近一周的天气','')


Beijing的天气预报：7
https://api.seniverse.com/v3/weather/daily.json?key=SUaEVEcUHIhRwQROw&location=Beijing&language=zh-Hans&unit=c&start=0&days=7
date ['2024-03-29', '2024-03-30', '2024-03-31']
weather_info {'date': ['2024-03-29', '2024-03-30', '2024-03-31'], 'forecast_day': ['晴', '晴', '晴'], 'forecast_night': ['晴', '晴', '晴'], 'maximum_temperature': ['20', '21', '23'], 'lowest_temperature': ['6', '6', '7']}


'根据最新的天气预报，接下来一周北京的天气以晴为主，白天最高气温分别为20℃、21℃、23℃，夜晚最低气温分别为6℃、6℃、7℃。请注意保暖和随时关注天气变化哦。如果您需要更多信息或有其他问题，请随时告诉我。'